# Strategy Backtesting
Testing the performance of sharpe based investment strategy.

## 1. Define Parameters

In [ ]:
import pandas as pd
import bt
import yfinance as yf
from src.metrics import sharpe_ratio
from src.strategy import create_weighted_balance_backtest

# Data Download
# stocks = ['CTAS','AWK','TDG','MKTX','DPZ','UNH','JKHY','ORLY','EXR','AVGO','SBUX','CPRT','MKC','AMT','RSG','VRSK','COO','AAPL','VRSN','LLY']  # Define your stock symbols
stocks = ['AAPL','BKNG','MNST','GILD','MCD','CHD','SBAC','ROP','TJX','SHW','COR','ACGL','TYL','WEC','AZO','WST','CTRA','ECL','HRL','ILMN','GIS']
start_dt = '2014-01-02'
end_dt = '2024-01-01'
interval = '1mo'

# Strategy Parameters
periods = 6

## 2. Download Ticker Data

In [ ]:

data = yf.download(stocks, start=start_dt, end=end_dt,interval=interval)
prices = data['Adj Close'].resample('MS').first()
prices.head()

## 3. Calculate Weight Values based on Sharpe

In [ ]:
returns = prices.pct_change(periods=periods).dropna()

sharpe = sharpe_ratio(returns, periods=periods)

weight_input = sharpe.dropna()
weight_input = weight_input.clip(lower=0)
weight_input = weight_input.div(weight_input.sum(axis=1), axis='index') 

print(weight_input.head())

data_input = prices.loc[weight_input.index.dropna()]

print(data_input.head())

df_plot = data_input / data_input.iloc[0]
df_plot.plot()

## 4. Create Strategy: Weighted vs Equal Weighting

In [ ]:
# Example usage
s1 = create_weighted_balance_backtest(data_input, weight_input)


# Function to create and run the baseline backtest with equal holdings
def equal_weight_backtest(data_input):
    # Define the strategy
    strategy = bt.Strategy('EqualWeight', 
                           [bt.algos.RunMonthly(),  # Run the strategy monthly
                            bt.algos.SelectAll(),   # Use all data provided
                            bt.algos.WeighEqually(),  # Assign equal weights to all assets
                            bt.algos.Rebalance()])  # Rebalance to target weights

    # Create a backtest object
    backtest = bt.Backtest(strategy, data_input)

    return backtest

# Example usage:
s_base = equal_weight_backtest(data_input)

## 5. Run and Plot Results

In [ ]:

# Running both strategies together
combined_result = bt.run(s1, s_base)

# Plotting cumulative returns to compare strategies
combined_result.plot(title='Strategy Comparison: Cumulative Returns')

# Displaying quantitative metrics
combined_result.display()

# For more detailed analysis, you can display and compare specific metrics like Sharpe Ratio, Max Drawdown, etc.
combined_result.display_lookback_returns()